### 1. Importing Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import urllib.request # library to open URLs

!pip -q install bs4
from bs4 import BeautifulSoup # import the BeautifulSoup library so we can parse HTML and XML documents

import json # library to handle JSON files

!pip -q install geopy #install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#RateLimiter, class which can be used to automatically add delays between geocoding calls to reduce the load on the Geocoding service. 
#Also it can retry failed requests and swallow errors for individual rows.
from geopy.extra.rate_limiter import RateLimiter 

import requests # library to handle requests

print('Libraries imported.')

Libraries imported.


In [2]:
#Bokeh and dependency libraries
!pip -q install bokeh
!pip -q install Jinja2 >=2.7
!pip -q install numpy >=1.11.3
!pip -q install packaging >=16.8
!pip -q install pillow >=4.0
!pip -q install python-dateutil >=2.1
!pip -q install PyYAML >=3.10
!pip -q install six >=1.5.2
!pip -q install tornado >=5
!pip -q install typing_extensions >=3.7.4

### 2. Get Toronto Dataset

##### Scrape Toronto Data from Wikipedia page.

In [3]:
# specify the URL/web page we are going to be scraping
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050"
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page)
print('Soup ready!')

Soup ready!


In [ ]:
# analyse the HTML underlying the website
#print(soup.prettify())

In [ ]:
# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
all_tables=soup.find_all("table")
#all_tables

In [4]:
right_table=soup.find('table', class_='wikitable sortable')
#right_table

# Loop through the rows

A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [5]:
#convert list to dataframe
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighborhood']=C
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


##### Explore Dataset and Data Cleaning.

In [6]:
print('Dataframe shape {}'.format(df.shape))
df.info()

Dataframe shape (287, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 3 columns):
PostalCode      287 non-null object
Borough         287 non-null object
Neighborhood    287 non-null object
dtypes: object(3)
memory usage: 6.8+ KB


In [7]:
print('Total Boroughs {}'.format(df['Borough'].drop_duplicates().reset_index(drop=True).count()))
df["Borough"].value_counts()

Total Boroughs 11


Not assigned        77
Etobicoke           45
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Name: Borough, dtype: int64

In [8]:
#Remove 'PostalCode' column.
df = df.drop(['PostalCode'],axis=1)

#Remove rows with Borough equals to 'Not assigned' and 'mississuaga'.
df = df[(df.Borough != 'Mississauga') & (df.Borough != 'Not assigned')].reset_index(drop=True)

print('Total Borough {}'.format(df['Borough'].drop_duplicates().reset_index(drop=True).count()))
print('Total Neighborhood {}'.format(df['Neighborhood'].count()))
print('Dataframe shape {}'.format(df.shape))
df.head()

Total Borough 9
Total Neighborhood 209
Dataframe shape (209, 2)


,Borough,Neighborhood
0,North York,Parkwoods
1,North York,Victoria Village
2,Downtown Toronto,Harbourfront
3,North York,Lawrence Heights
4,North York,Lawrence Manor


##### Get Neighborhood Latitude and Longitude using Geopy.

In [9]:
geolocator = Nominatim(user_agent="tor_explorer")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds = 10, error_wait_seconds =10)

# create empty lists
list_lat = []   
list_lng = []

for index, row in df.iterrows(): # iterate over rows in dataframe

    Neighborhood = row['Neighborhood']
    City = 'Toronto'     
    address = str(Neighborhood)+', '+ City
    #print(address)

    #location = geolocator.geocode(address)
    location = geocode(address)
    if location is None :
        lat = None
        lng = None
    else:
        lat = location.latitude
        lng = location.longitude
    #print('{},{}'.format(lat,lng))

    list_lat.append(lat)
    list_lng.append(lng)

# create new columns from lists    
df['Latitude'] = list_lat   
df['Longitude'] = list_lng

print('Dataframe shape {}'.format(df.shape))
df.head()

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Parkview Hill, Toronto',), **{}).
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/opt/conda/envs/Python36/lib/python3.6/http/client.py", line 1254, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/opt/conda/envs/Python36/lib/python3.6/http/client.py", line 1300, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/opt/conda/envs/Python36/lib/python3.6/http/client.py", line 1249, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/opt/conda/envs/Python36/lib/python3.6/http/client.py", line 1036, in _send_output
    self.send(msg)
  File "/opt/conda/envs/Python36/lib/python3.6/http/client.py", line 974, in send
    self.connect()
  File "/opt/conda/envs/Python36/lib/python3.6/htt

Dataframe shape (209, 4)


,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.758800,-79.320197
1,North York,Victoria Village,43.732658,-79.311189
2,Downtown Toronto,Harbourfront,43.640080,-79.380150
3,North York,Lawrence Heights,43.722778,-79.450933
4,North York,Lawrence Manor,43.722079,-79.437507


In [10]:
#remove rows with NaN values
toronto_data = df.dropna().reset_index(drop=True)
print('Dataframe shape {}'.format(toronto_data.shape))
toronto_data.head()

Dataframe shape (201, 4)


,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.758800,-79.320197
1,North York,Victoria Village,43.732658,-79.311189
2,Downtown Toronto,Harbourfront,43.640080,-79.380150
3,North York,Lawrence Heights,43.722778,-79.450933
4,North York,Lawrence Manor,43.722079,-79.437507


### 3. Explore Toronto Neighborhood 

##### Count Neighborhood of each Borough.

In [11]:
toronto_data['Borough'].value_counts()

Etobicoke           43
Scarborough         37
North York          37
Downtown Toronto    36
Central Toronto     17
West Toronto        13
East Toronto         6
York                 6
East York            6
Name: Borough, dtype: int64

In [12]:
# The code was removed by Watson Studio for sharing.

##### Search Mosques around Toronto Neighborhood using Foursquare API.

In [13]:
def getNearbyMosques(boroughs,neighborhoods, latitudes, longitudes):
    
    venues_list=[]
    for borough, neighborhood, lat, lng in zip(boroughs, neighborhoods, latitudes, longitudes):
        #print(neighborhoods)
            
        # create the API request URL
        LIMIT = 100 # limit of number of venues returned by Foursquare API
        radius = 10000 # define radius
        search_query = 'mosque'
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            VERSION, 
            lat, 
            lng, 
            search_query,
            radius, 
            LIMIT)
        #print(url)
         # make the GET request
        results = requests.get(url, timeout=10).json()["response"]['venues']
        #result = requests.get(url, timeout=20).json()
        #print('++++++++++++++++++++ {}'.format(result))
        #results = result["response"]['venues']
        
        #print(result)
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough,
            neighborhood, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng']) for v in results])

    nearby_mosques = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_mosques.columns = ['Boroughs',
                             'Neighborhoods',
                             'N_Lat',
                             'N_Lng',
                             'Mosques',
                             'M_Lat',
                             'M_Lng'] 
                  
    
    return(nearby_mosques)

In [14]:
toronto_mosques = getNearbyMosques(
    boroughs= toronto_data['Borough'],
    neighborhoods= toronto_data['Neighborhood'],
    latitudes= toronto_data['Latitude'],
    longitudes= toronto_data['Longitude']
    )

In [15]:
toronto_mosques = toronto_mosques.drop_duplicates(subset='Mosques', keep="last")
print(toronto_mosques.shape)
print('toronto_mosques shape {}'.format(toronto_mosques.shape))
toronto_mosques

(23, 7)
toronto_mosques shape (23, 7)


,Boroughs,Neighborhoods,N_Lat,N_Lng,Mosques,M_Lat,M_Lng
362,North York,York University,43.779242,-79.483559,Baitul Islam Mosque,43.865999,-79.534002
840,Central Toronto,North Midtown,43.162128,-77.619604,Hamidiye Mosque,43.160714,-77.566286
1080,Downtown Toronto,Railway Lands,49.266236,-123.237060,Ajyal Mosque | مسجد أجيال (Ajyal Mosque),49.279837,-123.108489
1081,Downtown Toronto,South Niagara,43.072145,-79.073868,Mosque Aisha,43.094220,-79.084564
1123,Etobicoke,Thistletown,43.737266,-79.565317,Vaughn Islamic Community Centre Mosque,43.828300,-79.537521
1125,Etobicoke,Thistletown,43.737266,-79.565317,Masjid E Saliheen,43.705980,-79.644635
1158,Scarborough,Upper Rouge,43.804930,-79.165837,Nugget mosque,43.780834,-79.232931
1159,Scarborough,Upper Rouge,43.804930,-79.165837,Jame Masjid Markham,43.841908,-79.264350
1180,Downtown Toronto,Underground city,43.770145,-79.374863,Abu Huraira Mosque Inc (Abu Huraira Center),43.772896,-79.334384
1183,Downtown Toronto,Underground city,43.770145,-79.374863,Süleymaniye Mosque Finch,43.763820,-79.482940


In [16]:
mosque_count = pd.DataFrame(toronto_mosques['Neighborhoods'].value_counts())
mosque_count = mosque_count.reset_index()
mosque_count.rename(columns={'index':'Neighborhoods','Neighborhoods':'NoofMosque'},inplace = True)
mosque_count

,Neighborhoods,NoofMosque
0,South of Bloor,8
1,Royal York South West,3
2,Underground city,3
3,Thistletown,2
4,Upper Rouge,2
5,Railway Lands,1
6,York University,1
7,South Niagara,1
8,The Queensway East,1
9,North Midtown,1


In [17]:
#merge dataframe
mosque_count = toronto_mosques.merge(mosque_count).drop(['N_Lat','N_Lng','Mosques','M_Lat','M_Lng'], axis=1)
mosque_count.head()

,Boroughs,Neighborhoods,NoofMosque
0,North York,York University,1
1,Central Toronto,North Midtown,1
2,Downtown Toronto,Railway Lands,1
3,Downtown Toronto,South Niagara,1
4,Etobicoke,Thistletown,2


##### Search Halal Restaurants Nearby Mosques using Foursquare API.

In [39]:
def getNearbyRestaurants(boroughs,neighborhoods, mosques, latitudes, longitudes):
    
    venues_list=[]
    for borough, neighborhood, mosque, lat, lng in zip(boroughs, neighborhoods, mosques, latitudes, longitudes):
                    
        # create the API request URL
        LIMIT = 100 # limit of number of venues returned by Foursquare API
        radius = 500 # define radius
        search_query = 'Halal Restaurant'
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            search_query,
            radius, 
            LIMIT)
        print(url)
            
         # make the GET request
        results = requests.get(url, timeout=10).json()["response"]['venues']
        #print(results)
        
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough,
            neighborhood,
            mosque,
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng']) for v in results])  

    nearby_resturants = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_resturants.columns = ['Boroughs', 
                                 'Neighborhoods',
                  'Mosques',
                  'M_Lat', 
                  'M_Lng', 
                  'Halal Restaurants', 
                  'R_Lat', 
                  'R_Lng'] 
                    
    return(nearby_resturants)

In [40]:
toronto_halal = getNearbyRestaurants(
    boroughs= toronto_mosques['Boroughs'],
    neighborhoods= toronto_mosques['Neighborhoods'],
    mosques= toronto_mosques['Mosques'],
    latitudes= toronto_mosques['M_Lat'],
    longitudes= toronto_mosques['M_Lng']
    )

https://api.foursquare.com/v2/venues/search?&client_id=CXOP2GJYWTVV2JCPAWWRYKSYBLYQXG4KFAYLOQSHZDTP54KN&client_secret=4CJHJO1SBF4LOAPCYIIMQ44YKHKMOZIYMCGB3IAMPB20LSIQ&v=20180605&ll=43.86599899522713,-79.53400176224416&query=Halal Restaurant&radius=500&limit=100
https://api.foursquare.com/v2/venues/search?&client_id=CXOP2GJYWTVV2JCPAWWRYKSYBLYQXG4KFAYLOQSHZDTP54KN&client_secret=4CJHJO1SBF4LOAPCYIIMQ44YKHKMOZIYMCGB3IAMPB20LSIQ&v=20180605&ll=43.160714,-77.566286&query=Halal Restaurant&radius=500&limit=100
https://api.foursquare.com/v2/venues/search?&client_id=CXOP2GJYWTVV2JCPAWWRYKSYBLYQXG4KFAYLOQSHZDTP54KN&client_secret=4CJHJO1SBF4LOAPCYIIMQ44YKHKMOZIYMCGB3IAMPB20LSIQ&v=20180605&ll=49.27983726882993,-123.10848919115963&query=Halal Restaurant&radius=500&limit=100
https://api.foursquare.com/v2/venues/search?&client_id=CXOP2GJYWTVV2JCPAWWRYKSYBLYQXG4KFAYLOQSHZDTP54KN&client_secret=4CJHJO1SBF4LOAPCYIIMQ44YKHKMOZIYMCGB3IAMPB20LSIQ&v=20180605&ll=43.09422,-79.084564&query=Halal Restaurant&radiu

In [20]:
toronto_halal = toronto_halal.drop_duplicates(subset='Halal Restaurants', keep="last")
print(toronto_halal.shape)
toronto_halal.head()

(130, 8)


,Boroughs,Neighborhoods,Mosques,M_Lat,M_Lng,Halal Restaurants,R_Lat,R_Lng
0,Central Toronto,North Midtown,Hamidiye Mosque,43.160714,-77.566286,Carroll's,43.160017,-77.563975
1,Central Toronto,North Midtown,Hamidiye Mosque,43.160714,-77.566286,Jamaican Restaurant,43.160168,-77.570656
2,Central Toronto,North Midtown,Hamidiye Mosque,43.160714,-77.566286,Chang Lung Chinese Restaurant,43.163400,-77.563753
3,Central Toronto,North Midtown,Hamidiye Mosque,43.160714,-77.566286,"Win Wah Chinese Restaurant, Rochester",43.159832,-77.559790
4,Downtown Toronto,Railway Lands,Ajyal Mosque | مسجد أجيال (Ajyal Mosque),49.279837,-123.108489,Yagger's Downtown Restaurant & Sports Bar,49.283161,-123.112503


In [21]:
halal_count = pd.DataFrame(toronto_halal['Mosques'].value_counts())
halal_count = halal_count.reset_index()
halal_count.rename(columns={'index':'Mosques','Mosques':'NoofHalalRestaurant'}, inplace = True)
halal_count

,Mosques,NoofHalalRestaurant
0,Toronto Mosque,38
1,Ajyal Mosque | مسجد أجيال (Ajyal Mosque),30
2,Hamza Mosque,9
3,Jame Abu Bakr Siddique Masjid,7
4,Baitul Aman Mosque,6
5,Mosque Aisha,6
6,Süleymaniye Mosque Finch,5
7,Mosque,5
8,Masjid E Saliheen,5
9,Hamidiye Mosque,4


In [22]:
#merge dataframe
halal_count = toronto_halal.merge(halal_count).drop(['M_Lat','M_Lng','Halal Restaurants','R_Lat','R_Lng'], axis=1)
halal_count.head()

,Boroughs,Neighborhoods,Mosques,NoofHalalRestaurant
0,Central Toronto,North Midtown,Hamidiye Mosque,4
1,Central Toronto,North Midtown,Hamidiye Mosque,4
2,Central Toronto,North Midtown,Hamidiye Mosque,4
3,Central Toronto,North Midtown,Hamidiye Mosque,4
4,Downtown Toronto,Railway Lands,Ajyal Mosque | مسجد أجيال (Ajyal Mosque),30


##### Search markets, groceries store, etc nearby Mosque around Neighborhood using Foursquare API.

In [91]:
def getNearbyMarkets(boroughs, neighborhoods, mosques, latitudes, longitudes):
    
    markets_list=[]
    for borough, neighborhood, mosque, lat, lng in zip(boroughs, neighborhoods, mosques, latitudes, longitudes):
    
            
        # create the API request URL
        LIMIT = 100 # limit of number of venues returned by Foursquare API
        radius = 500 # define radius
        search_query = ['Farmers Market','Fish Market','Grocery Store']
        
        for i in search_query :
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                i,
                radius, 
                LIMIT)
            #print(url)
        
            
         # make the GET request
            results = requests.get(url, timeout=10).json()["response"]['venues']
        #print(results)

        # return only relevant information for each nearby venue
            markets_list.append([(
                borough, 
                neighborhood,
                mosque,
                lat, 
                lng, 
                v['name'], 
                v['location']['lat'], 
                v['location']['lng'])for v in results])
        
        search_query=search_query[+1:+1]
       
    
    nearby_markets = pd.DataFrame([item for market_list in markets_list for item in market_list])
    nearby_markets.columns = [
        'Boroughs',
        'Neighborhoods',
        'Mosques',
        'M_Lat',
        'M_Lng',
        'Markets',
        'Mr_Lat',
        'Mr_Lng'] 
                  
    
    return(nearby_markets)

In [92]:
toronto_market = getNearbyMarkets(
    boroughs= toronto_mosques['Boroughs'],
    neighborhoods= toronto_mosques['Neighborhoods'],
    mosques= toronto_mosques['Mosques'],
    latitudes= toronto_mosques['M_Lat'],
    longitudes= toronto_mosques['M_Lng']
    )

In [93]:
toronto_market = toronto_market.drop_duplicates(subset='Markets', keep="last")
print(toronto_market.shape)
toronto_market.head()

(197, 8)


,Boroughs,Neighborhoods,Mosques,M_Lat,M_Lng,Markets,Mr_Lat,Mr_Lng
0,North York,York University,Baitul Islam Mosque,43.865999,-79.534002,Ocean Fox Fishing and Diving,43.868585,-79.538687
4,Central Toronto,North Midtown,Hamidiye Mosque,43.160714,-77.566286,The Insurance Market Place,43.162029,-77.565576
5,Central Toronto,North Midtown,Hamidiye Mosque,43.160714,-77.566286,Ariana Super Market,43.164597,-77.563972
6,Central Toronto,North Midtown,Hamidiye Mosque,43.160714,-77.566286,Cal's Liquor,43.164074,-77.564241
7,Central Toronto,North Midtown,Hamidiye Mosque,43.160714,-77.566286,Bernie Sanders Store,43.162435,-77.565146


In [94]:
market_count = pd.DataFrame(toronto_market['Mosques'].value_counts())
market_count = market_count.reset_index()
market_count.rename(columns={'index':'Mosques','Mosques':'NoofMarket'}, inplace = True)
market_count

,Mosques,NoofMarket
0,Toronto Mosque,57
1,Ajyal Mosque | مسجد أجيال (Ajyal Mosque),54
2,Vaughn Islamic Community Centre Mosque,26
3,Hamza Mosque,15
4,Bosnian Mosque Birmingham,10
5,Hamidiye Mosque,8
6,Baitul Aman Mosque,5
7,Albanian Mosque - Albanian Muslim Society of T...,4
8,Mosque Aisha,3
9,Abu Huraira Mosque Inc (Abu Huraira Center),3


In [95]:
#merge dataframe
market_count = toronto_market.merge(market_count).drop(['Boroughs','M_Lat','M_Lng','Markets','Mr_Lat','Mr_Lng'], axis=1)
market_count.head()

,Neighborhoods,Mosques,NoofMarket
0,York University,Baitul Islam Mosque,1
1,North Midtown,Hamidiye Mosque,8
2,North Midtown,Hamidiye Mosque,8
3,North Midtown,Hamidiye Mosque,8
4,North Midtown,Hamidiye Mosque,8


In [96]:
toronto_market.groupby(['Neighborhoods', 'Mosques']).agg(['count'])

Boroughs  \
                                                                             count   
Neighborhoods          Mosques                                                       
North Midtown                                                                    8   
         Hamidiye Mosque                                                        54   
Railway Lands          Ajyal Mosque | مسجد أجيال (Ajyal Mosque)                  1   
Royal York South West                                                            1   
 Baitul Hamd Mosque                                                              3   
                       Ummah Nabawiah Mosque                                     4   
South Niagara          Mosque Aisha                                              5   
South of Bloor                                                                  10   
        Albanian Mosque - Albanian Muslim Society of To...                      15   
                       Baitul Aman Mosque                                        1   
                       Bosnian Mosque Birmingham                                 1   
                       Hamza Mosque                                              1   
                       Mevlana Mosque                                           57   
                       Mosque                                                    1   
                       Omar Bin Khattab Mosque                                  26   
                       Toronto Mosque                                            3   
The Queensway East     Istiqamah Islamic Centre of Ontario                       2   
Thistletown            Vaughn Islamic Community Centre Mosque                    3   
Underground city       Abu Huraira Mosque Inc (Abu Huraira Center)               1   
                       Jame Abu Bakr Siddique Masjid                                 
Upper Rouge            Nugget mosque                                                 
York University        Baitul Islam Mosque                                           

                                                                          M_Lat  \
                                                                          count   
Neighborhoods          Mosques                                                    
North Midtown                                                                 8   
         Hamidiye Mosque                                                     54   
Railway Lands          Ajyal Mosque | مسجد أجيال (Ajyal Mosque)               1   
Royal York South West                                                         1   
 Baitul Hamd Mosque                                                           3   
                       Ummah Nabawiah Mosque                                  4   
South Niagara          Mosque Aisha                                           5   
South of Bloor                                                               10   
        Albanian Mosque - Albanian Muslim Society of To...                   15   
                       Baitul Aman Mosque                                     1   
                       Bosnian Mosque Birmingham                              1   
                       Hamza Mosque                                           1   
                       Mevlana Mosque                                        57   
                       Mosque                                                 1   
                       Omar Bin Khattab Mosque                               26   
                       Toronto Mosque                                         3   
The Queensway East     Istiqamah Islamic Centre of Ontario                    2   
Thistletown            Vaughn Islamic Community Centre Mosque                 3   
Underground city       Abu Huraira Mosque Inc (Abu Huraira Center)            1   
                       Jame Abu Bakr Siddique Masjid                              
Upper Rouge   

In [101]:
#toronto_market['Neighborhoods' == 'South of Bloom'] 
toronto_market.loc[toronto_market['Mosques'] == 'Hamza Mosque']

,Boroughs,Neighborhoods,Mosques,M_Lat,M_Lng,Markets,Mr_Lat,Mr_Lng
366,Etobicoke,South of Bloor,Hamza Mosque,43.639717,-79.432343,Independent City Market,43.639071,-79.427563
367,Etobicoke,South of Bloor,Hamza Mosque,43.639717,-79.432343,Tang's Market,43.641688,-79.432156
368,Etobicoke,South of Bloor,Hamza Mosque,43.639717,-79.432343,Hasty Market,43.638866,-79.427539
369,Etobicoke,South of Bloor,Hamza Mosque,43.639717,-79.432343,Super Land Market,43.641156,-79.434088
370,Etobicoke,South of Bloor,Hamza Mosque,43.639717,-79.432343,Queen Fresh Market,43.641308,-79.433740
371,Etobicoke,South of Bloor,Hamza Mosque,43.639717,-79.432343,King's Food Market,43.638688,-79.428782
372,Etobicoke,South of Bloor,Hamza Mosque,43.639717,-79.432343,Bala Super Market,43.640435,-79.436935
373,Etobicoke,South of Bloor,Hamza Mosque,43.639717,-79.432343,Cattlemen's Meat Market,43.639982,-79.439370
374,Etobicoke,South of Bloor,Hamza Mosque,43.639717,-79.432343,Thinkbox National Marketing,43.636739,-79.426212
376,Etobicoke,South of Bloor,Hamza Mosque,43.639717,-79.432343,Dakao Grocery,43.641670,-79.430947


### 4. Data Visualisation

In [68]:
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral5, Spectral10, Category20
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
#from bokeh.resources import INLINE
#output_notebook(resources=INLINE)
output_notebook()

Loading BokehJS ...

##### How many mosque(s) within 5 kilometers of the neighborhood from each borough ?

In [ ]:
group = mosque_count.groupby(by=['Boroughs','Neighborhoods'])

index_cmap = factor_cmap('Boroughs_Neighborhoods', palette=Spectral5, factors=sorted(mosque_count.Boroughs.unique()), end=1)

m = figure(plot_width=800, title="No. of Mosque by Neighborhood and Borough",
           x_range=group, toolbar_location=None, tooltips=[("NoofMosque", "@NoofMosque_mean")])

m.vbar(x='Boroughs_Neighborhoods', top='NoofMosque_mean', width=1, source=group,
       line_color="white", fill_color=index_cmap)

m.y_range.start = 0
m.x_range.range_padding = 0.05
m.xgrid.grid_line_color = None
m.xaxis.axis_label = "Neighborhood, Borough"
m.yaxis.axis_label = "No. of Mosque"
m.xaxis.major_label_orientation = 1.57
m.outline_line_color = None

show(m)

##### How many halal restaurants within 500 meters from each mosque?

In [99]:
#grp = halal_count.groupby(by=['Boroughs','Mosques'])
#nbhd = list(group.Neighborhoods)
#source = ColumnDataSource(data=dict(group = grp,neighborhood = nbhd))

group = halal_count.groupby(by=['Neighborhoods','Mosques'])
#print('{}'.format(group.shape()))
index_cmap = factor_cmap('Neighborhoods_Mosques', palette=Spectral9, factors=sorted(halal_count.Neighborhoods.unique()), end=1)

h = figure( title="No. of Halal Restaurant by Mosque and Borough",
           x_range=group, toolbar_location=None, tooltips=[("NoofHalalRestaurant", "@NoofHalalRestaurant_mean")])

h.vbar(x='Neighborhoods_Mosques', top='NoofHalalRestaurant_mean', width=1, source=group,
       line_color="white", fill_color=index_cmap)
#print('{}'.format(group[1]['Neigborhoods']))
h.y_range.start = 0
h.x_range.range_padding = 0.05
h.xgrid.grid_line_color = None
h.xaxis.axis_label = "Mosque, Neighborhood"
h.yaxis.axis_label = "No. of Restaurant Nearby Mosques"
h.xaxis.major_label_orientation = 1.57
h.xaxis.group_label_orientation =1.57
h.outline_line_color = None

show(h)

##### How many markets within 500 meters from each mosques?

In [98]:
group = market_count.groupby(by=['Neighborhoods','Mosques'])

index_cmap = factor_cmap('Neighborhoods_Mosques', palette=Spectral9, factors=sorted(market_count.Neighborhoods.unique()), end=1)

mr = figure( title="No. of Markets Nearby Mosque",
           x_range=group, toolbar_location=None, tooltips=[("NoofMarket", "@NoofMarket_mean")])

mr.vbar(x='Neighborhoods_Mosques', top='NoofMarket_mean', width=1, source=group,
       line_color="white", fill_color=index_cmap)

mr.y_range.start = 0
mr.x_range.range_padding = 0.05
mr.xgrid.grid_line_color = None
mr.xaxis.axis_label = "Mosque, Neighborhood"
mr.yaxis.axis_label = "No. of Market Nearby Mosques"
mr.xaxis.major_label_orientation = 1.57
mr.xaxis.group_label_orientation =1.57
mr.outline_line_color = None

show(mr)

In [ ]:
from IPython.display import HTML
import base64 

def create_download_link( df, title = "Download CSV file", filename = "x.csv"):  
    csv = x.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(df)